<a href="https://colab.research.google.com/github/Jasleen8801/medical-prescription-recognition/blob/main/medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
!pip install opencv-python
!pip install opencv-python-headless
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay
!pip install wget
!pip install emnist

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-xkb-utils
  xfonts-base xfonts-encodings xfonts-utils xserver-common
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-utils
  x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 12 newly installed, 0 to remove and 15 not upgraded.
Need to get 7,920 kB of archives.
After this operation, 12.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libfontenc1 amd64 1:1.1.4-0ubuntu1 [14.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libxfont2 amd64 1:2.0.3-1 [91.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libxkbfile1 amd64 1:1.1.0-1 [65.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu f

In [4]:
# Import necessary libraries
import numpy as np
from PIL import ImageGrab, ImageTk, Image
from google.colab import files
import tkinter as tk
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
import cv2
import time
import wget
import pandas as pd
from scipy.io import loadmat
import emnist
from scipy.optimize import minimize

In [5]:
# Functions for training the neural network
def neural_network(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lamb):
  # weights split to Theta1 and Theta2
  Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                      hidden_layer_size, input_layer_size + 1)
  Theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],
                      (num_labels, hidden_layer_size+1))

  # Forward Propagation
  m = X.shape[0]  # no of training examples
  one_matrix = np.ones((m,1))
  X = np.append(one_matrix, X, axis=1) # Adding bias unit to first layer by appending ones to the left of X
  a1 = X # Activation of 1st layer = X
  z2 = np.dot(X, Theta1.transpose()) # Product of X and transpose of theta
  a2 = 1 / (1 + np.exp(-z2)) # Apply the sigmoid function to z2 to compute the activation of the second layer
  one_matrix = np.ones((m,1))
  a2 = np.append(one_matrix, a2, axis=1)
  z3 = np.dot(a2, Theta2.transpose())
  a3 = 1 / (1 + np.exp(-z3))

  # Changing y labels into vectors of bool
  y_vect = np.zeros((m, num_labels))
  for i in range(m):
    y_vect[i, int(y[i])] = 1

  # Calculating cost function
  J = (1 / m) * (np.sum(np.sum(-y_vect * np.log(a3) - (1 - y_vect) * np.log(1 - a3)))) + (lamb / (2 * m)) * (
                np.sum(np.sum(Theta1[:, 1:] ** 2)) + np.sum(np.sum(Theta2[:, 1:] ** 2)))

  # Backpropagation
  Delta3 = 3 - y_vect # error in o/p layer
  Delta2 = np.dot(Delta3, Theta2) * a2 * (1-a2) # error in hidden layer
  Delta2 = Delta2[:, 1:] # Remove the error for the bias unit in the hidden layer

  # Gradient Computation
  Theta1[:,0] = 0
  Theta1_grad = (1/m) * np.dot(Delta2.transpose(), a1) + (lamb/m) * Theta1
  Theta2[:,0] = 0
  Theta2_grad = (1/m) * np.dot(Delta3.transpose(), a2) + (lamb/m) * Theta2
  grad = np.concatenate((Theta1_grad.flatten(), Theta2_grad.flatten()))

  return J, grad

In [6]:
# Function for predicting the output
def predict(Theta1, Theta2, X):
  m = X.shape[0]
  one_matrix = np.ones((m, 1))
  X = np.append(one_matrix, X, axis=1)
  z2 = np.dot(X, Theta1.transpose())
  a2 = 1 / (1 + np.exp(-2))
  one_matrix = np.ones((m, 1))
  a2 = np.append(one_matrix, a2, axis=1)
  z3 = np.dot(a2, Theta2.transpose())
  a3 = 1 / (1 + np.exp(-z3))
  p = np.argmax(a3, axis=1)
  return p

In [7]:
def initialise(a, b):
  epsilon = 0.15
  c = np.random.rand(a, b+1)*(2*epsilon)-epsilon
  return c

In [8]:
# Function to capture screen portion
def capture_screen(x, y, width, height):
  display(Javascript("""
    const sleep = (milliseconds) => {
      return new Promise(resolve => setTimeout(resolve, milliseconds))
    }
    async function capture(x, y, width, height) {
      let kernel = IPython.notebook.kernel;
      let result = await google.colab.kernel.invokeFunction('notebook.capture', [
        x, y, width, height
      ], {});
      let img_data = result.data['application/octet-stream'];
      const urlCreator = window.URL || window.webkitURL;
      const imageURL = urlCreator.createObjectURL(new Blob([img_data], {type; "image/jpeg"}));
      const img = document.createElement('img');
      img.src = imageURL;
      img.width = width;
      img.height = height;
      document.body.appendChild(img);
      await sleep(1000);
      img.remove();
    }
    capture({}, {}, {}, {})
  """))

In [9]:
# function to recognise prescription
def recognize_prescription():
  x = 100
  y = 100
  width = 500
  height = 500

  capture_screen(x, y, width, height)

  img = cv2.imread("screenshot.jpg")
  cv2.imshow(img)


In [10]:
wget.download('https://www.kaggle.com/datasets/crawford/emnist?select=emnist-balanced-train.csv', 'emnist-balanced-train.csv')
wget.download('https://www.kaggle.com/datasets/crawford/emnist?select=emnist-balanced-test.csv', 'emnist-balanced-test.csv')

'emnist-balanced-test.csv'

In [11]:
# data1 = pd.read_csv('emnist-balanced-train.csv')
# data2 = pd.read_csv('emnist-balanced-test.csv')
X_train, y_train = emnist.extract_training_samples('balanced')
X_test, y_test = emnist.extract_test_samples('balanced')

In [19]:
X_train = X_train.reshape((112800, 784), order='F')
X_test = X_test.reshape((18800, 784), order='F')

X_train = X_train / 255.0
X_test = X_test / 255.0

for i in range(len(X_train)):
  ex = X_train[i,:].reshape((28,28), order='F')  # Use 'F' for column-major order
  X_train[i,:] = ex.flatten()

for i in range(len(X_test)):
  ex = X_test[i,:].reshape((28,28), order='F')  # Use 'F' for column-major order
  X_test[i,:] = ex.flatten()


In [20]:
y_train = y_train.flatten()
y_test = y_test.flatten()

In [21]:
m = X_train.shape[0]
input_layer_size = 784
hidden_layer_size = 130
num_labels = 47

# randomly initialising thetas
initial_theta1 = initialise(hidden_layer_size, input_layer_size)
initial_theta2 = initialise(num_labels, hidden_layer_size)

# unrolling parameters into a single column vector
initial_nn_params = np.concatenate((initial_theta1.flatten(), initial_theta2.flatten()))
maxiter = 800
lambda_reg = 0.1
myargs = (input_layer_size, hidden_layer_size, num_labels, X_train, y_train, lambda_reg)

# calling minimize function to minimize cost function and to train weights
results = minimize(neural_network, x0=initial_nn_params, args=myargs, options={
    'disp': True,
    'maxiter': maxiter
}, method="L-BFGS-B", jac=True)

nn_params = results["x"]

# Weights are split back to theta1 and theta2
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, input_layer_size + 1))  # shape = (100, 785)
Theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],
                    (num_labels, hidden_layer_size + 1))  # shape = (10, 101)

pred = predict(Theta1, Theta2, X_test)
print(f'Test Set Accuracy: {(np.mean(pred == y_test) * 100)}')

pred = predict(Theta1, Theta2, X_train)
print(f'Training Set Accuracy: {(np.mean(pred == y_train) * 100)}')

tp = 0
for i in range(len(pred)):
  if pred[i] == y_train[i]:
    tp += 1
fp = len(y_train) - tp
print(f'Precision: {tp/(tp+fp)}')

np.savetxt('Theta1.txt', Theta1, delimiter=',')
np.savetxt('Theta2.txt', Theta2, delimiter=',')

TypeError: ignored